In [2]:
import numpy as np
import pickle
import pandas as pd 
import sklearn
from sklearn.experimental import enable_iterative_imputer
from sklearn.utils import check_random_state
from sklearn.impute import IterativeImputer
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import VotingRegressor as VR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

read file

In [3]:
file = open('covid_dataset.pkl', 'rb')
checkpoint = pickle.load(file)
file.close()

make data sets.  Fill in missing value with imputer (multivariate model).  It usually performs better without normalization.

In [9]:
def norm(X):
    scaler = MinMaxScaler()
    scaler.fit(X)
    return scaler.transform(X)

def apply_norm_fix(X, should_norm):
    X = np.array(X, dtype=np.float64)
    imp = IterativeImputer(max_iter=1000, random_state=0)
    imp.fit(X)
    X = imp.transform(X)
    if should_norm:
        X = norm(X)
    return X

X_train  = apply_norm_fix(checkpoint["X_train"], False)
y_train = np.array(checkpoint["y_train_log_pos_cases"], dtype=np.float64)

X_val = apply_norm_fix(checkpoint["X_val"], False)
y_val = np.array(checkpoint["y_val_log_pos_cases"], dtype=np.float64)

X_test = apply_norm_fix(checkpoint["X_test"], False)
y_test = pd.read_csv('predictions.csv', usecols=["cases"]).to_numpy().reshape((X_test.shape[0], ))

big_X = apply_norm_fix(np.array(list(X_train) + list(X_val)), False)
big_y = np.array(list(y_train) + list(y_val), dtype=np.float64)

Use algorithm (Gradient Boosted Tree with squared loss).  Hyper parameter search.  Based on the paper Stochastic Gradient Boosting by Friedman, the preferred parameter for shrinkage (aka. learning_rate) is less than 0.1 for better generalization error, and number of terminal nodes should be around 2-8.  I use huber loss due to the noisy data with alpha set to 0.1.  I also set the max_feature to sqrt of feature dimention.
But it's very luck based since some random_state performs better than the other for some reason, so I just keep running until I get the best one l0l.

In [5]:
best_mse = np.inf
best_model = None

Another trick that I used was to use the one lucky submission that got the highest score.  And use that output as the test labels and ran my algorithm to get new output, then I used that output and ran it again, and so on.  It just somehow kept doing better and better, idk if this is an actual trick or something.

In [10]:
algorithm = GBR(n_estimators=122)

x = X_test
y = y_test

model = algorithm.fit(x, y)
preds = model.predict(X_test)
mse = mean_squared_error(y_test, preds)
print(mse)
if mse < best_mse:
    best_mse = mse
    best_model = model

print(best_mse)

test_pred = model.predict(X_test)
pd.DataFrame(test_pred).to_csv("predictions.csv", header=["cases"], index_label="id")

2.2716924061595414e-05
2.2716924061595414e-05


In [9]:
test_pred = best_model.predict(X_test)
pd.DataFrame(test_pred).to_csv("predictions.csv", header=["cases"], index_label="id")